In [ ]:
from dataclasses import dataclass
import json
import logging
import os

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import simplekv.fs
from skimage.measure import block_reduce
from skimage.transform import resize
import yaml

from cacheutil import CacheEntry, make_key


logging.basicConfig(level=logging.INFO)
logging.getLogger("cacheutil").setLevel(logging.DEBUG)

In [ ]:
# 3 ways to run the notebook (uncomment one of the following)

# 1) interactive mode with inline parameters
# MAP_NAME = None

# 2) interactive mode with parameters from YAML file
# 3) batch mode
MAP_NAME = os.environ.get("MAP_NAME", "tokyo-dos")

# batch mode: run from the terminal like this:
# MAP_NAME=delft jupyter nbconvert --execute --to notebook extract_map.ipynb

if MAP_NAME:
    with open(f"maps/{MAP_NAME}.yaml") as f:
        MAP_INFO = yaml.safe_load(f)
    # print(MAP_INFO)
else:
    MAP_INFO = {}

cache = simplekv.fs.FilesystemStore(root="_cache")

In [ ]:
def discretize_extent(tile_extent: tuple[tuple[float, float], tuple[float, float]],
                      tile_shape: tuple[int, int],
                      requested_extent: tuple[tuple[float, float], tuple[float, float]]):
    # axis 0 is latitude north->south (!)
    # axis 1 is longitude west->east

    (tile_min_lat, tile_min_lon), (tile_max_lat, tile_max_lon) = tile_extent
    relative_extent = ((requested_extent[0][0] - tile_min_lat, requested_extent[0][1] - tile_min_lon),
                       (requested_extent[1][0] - tile_min_lat, requested_extent[1][1] - tile_min_lon))

    print(f"{relative_extent=}")

    # project user extents into pixel space
    px_per_deg_lat = tile_shape[0] / (tile_max_lat - tile_min_lat)
    px_per_deg_lon = tile_shape[1] / (tile_max_lon - tile_min_lon)

    print(f"{px_per_deg_lat=} {px_per_deg_lon=}")

    min_y = int(np.floor(relative_extent[0][0] * px_per_deg_lat))
    min_x = int(np.floor(relative_extent[0][1] * px_per_deg_lon))
    max_y = int(np.ceil(relative_extent[1][0] * px_per_deg_lat))
    max_x = int(np.ceil(relative_extent[1][1] * px_per_deg_lon))

    print(f"{min_y=} {min_x=} {max_y=} {max_x=}")
    assert min_y >= 0 and min_y <= tile_shape[0]
    assert min_x >= 0 and min_x <= tile_shape[1]
    assert max_y >= 0 and max_y <= tile_shape[0]
    assert max_x >= 0 and max_x <= tile_shape[1]

    # reconstruct geo coordinates
    discretized_extent = ((tile_min_lat + min_y / px_per_deg_lat, tile_min_lon + min_x / px_per_deg_lon),
                          (tile_min_lat + max_y / px_per_deg_lat, tile_min_lon + max_x / px_per_deg_lon))
    print(f"{discretized_extent=}")

    # flip Y
    min_y_corr = tile_shape[0] - max_y
    max_y_corr = tile_shape[0] - min_y
    print(f"{min_y_corr=} {max_y_corr=}")
    assert min_y_corr >= 0 and min_y_corr <= tile_shape[0]
    assert max_y_corr >= 0 and max_y_corr <= tile_shape[0]

    return ((min_y_corr, max_y_corr), (min_x, max_x)), discretized_extent

In [ ]:
def load_unzipped(coords):
    # dataset is provided in the form of tiles 3x3 degrees in size
    # compute tile coordinates
    tile_coords = (int((coords[0] // 3) * 3), int((coords[1] // 3) * 3))

    tile_extent = (tile_coords, (tile_coords[0] + 3, tile_coords[1] + 3))

    # Build macrotile archive name from macrotile coords.
    # Archive naming uses hemisphere prefixes: N30E120 etc. Determine strings:
    lat_prefix = f"N{abs(tile_coords[0]):02d}" if tile_coords[0] >= 0 else f"S{abs(tile_coords[0]):02d}"
    lon_prefix = f"E{int(tile_coords[1]):03d}" if tile_coords[1] < 180 else f"W{int((360-tile_coords[1])%360):03d}"

    filename = f"inputs/ESA_WorldCover_10m_2021_V200_{lat_prefix}{lon_prefix}_Map.tif"
    print("load", filename)

    with rasterio.open(filename) as dataset:
        data = dataset.read(1)  # read first band
        profile = dataset.profile
    return data, profile, tile_extent


center_coords = MAP_INFO.get("center_coords", (35.48556194027054, 139.86056640390504))
tile_data, profile, tile_extent = load_unzipped(center_coords)

print(f"{tile_data.shape=}; {tile_data.nbytes / (1024 * 1024):.0f} MB")
print(f"{profile=}")
print(f"{tile_extent=}")

In [ ]:
# extract region
extents = MAP_INFO.get("extents", ((34.75, 139.0), (36, 141)))
coords, discretized_extent = discretize_extent(tile_extent=tile_extent,
                                               tile_shape=tile_data.shape,
                                               requested_extent=extents)

# extract just a small area of the 36000x36000 image for display
# axis 0 is north->south (before we flip it)
# axis 1 is west->east
extract_data1 = tile_data[coords[0][0]:coords[0][1], coords[1][0]:coords[1][1]][::-1]

# initial round of decimation; first calculate decimation factor to get close to goal width
GOAL_W = 46
GOAL_H = int(GOAL_W * extract_data1.shape[0] / extract_data1.shape[1])

# this will be the (square root of) number of samples per cell in the final map
OVERSAMPLE = 5

# use nearest-neighbor scaling
extract_data = resize(extract_data1, (GOAL_H * OVERSAMPLE, GOAL_W * OVERSAMPLE),
                      order=0, anti_aliasing=False, preserve_range=True)
print(f"{extract_data.shape=} {extract_data.dtype=}")


f, ax = plt.subplots(figsize=(10, 10))
cmap = ListedColormap([
    "#000000",  # No data
    "#006400",  # Tree cover
    "#ffbb22",  # Shrubland
    "#ffff4c",  # Grassland
    "#f096ff",  # Cropland
    "#fa0000",  # Built-up
    "#b4b4b4",  # Bare / sparse vegetation
    "#f0f0f0",  # Snow and ice
    "#0064c8",  # Permanent water bodies
    "#0096a0",  # Herbaceous wetland
    "#00cf75",  # Mangroves
    "#fae6a0",  # Moss and lichen
])
im = ax.imshow(extract_data // 10, origin="lower", cmap=cmap, vmin=0, vmax=11)
cbar = f.colorbar(im, ax=ax, fraction=0.036, pad=0.04, ticks=np.arange(0, 12))
cbar.ax.set_yticklabels([
    "No data",
    "Tree cover",
    "Shrubland",
    "Grassland",
    "Cropland",
    "Built-up",
    "Bare / sparse vegetation",
    "Snow and ice",
    "Permanent water bodies",
    "Herbaceous wetland",
    "Mangroves",
    "Moss and lichen",
])
None

In [ ]:
NO_DATA = 0
WATER = 80
water_mask_large = np.logical_or(extract_data == NO_DATA, extract_data == WATER).astype(np.uint8)

f, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(water_mask_large, origin="lower", cmap="Blues")

In [ ]:
# use this trick: https://stackoverflow.com/a/26639111
assert water_mask_large.shape[0] % OVERSAMPLE == 0
assert water_mask_large.shape[1] % OVERSAMPLE == 0

water_mask_decimated = block_reduce(water_mask_large, block_size=OVERSAMPLE, func=np.mean)
print(f"{water_mask_decimated.shape=}")

f, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(water_mask_decimated, origin="lower", cmap="Blues")

#
water_threshold = MAP_INFO.get("water_threshold", 0.50)
water_mask = (water_mask_decimated >= water_threshold).astype(np.uint8)

f, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(water_mask, origin="lower", cmap="Blues")

In [ ]:
with CacheEntry(cache, make_key("height_raw", discretized_extent), type="numpy") as cached:
    if cached.has_data():
        height_data = cached.get()
    else:
        import sys
        sys.path.insert(0, "/home/mce/dev/CLOU25/")
        import elevmodel

        import logging

        logging.basicConfig()
        logging.getLogger("elevmodel").setLevel(logging.DEBUG)

        # model = elevmodel.NASADEM_HGT("/mnt/nas/public/Datasets/NASADEM_HGT.001", format="zip")
        model = elevmodel.SRTMGL1("/mnt/nas/public/Datasets/SRTMGL1", format="zip")

        height_data, _, _, _, _ = elevmodel.extract(model, *discretized_extent[0], *discretized_extent[1])

        cached.put(height_data)

In [ ]:
with CacheEntry(cache, make_key("height_data_resized", discretized_extent, water_mask.shape), type="pickle") as cached:
    if cached.has_data():
        height_data_resized = cached.get()
    else:
        # ~20sec to compute
        height_data_resized = resize(height_data, water_mask.shape, preserve_range=True, order=1, mode="edge", anti_aliasing=True)

        cached.put(height_data_resized)

height = height_data_resized.copy()
height[water_mask > 0] = -100

# plot height_data_resized
f, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(height, origin="lower")
cbar = f.colorbar(im, ax=ax, fraction=0.036, pad=0.04)
cbar.ax.set_ylabel("Height (m)")

In [ ]:
with CacheEntry(cache, make_key(MAP_INFO["country_code"], discretized_extent), type="json") as cached:
    if cached.has_data():
        cities = cached.get()
    else:
        import extract_cities
        # acquire data like this:
        # curl -O https://download.geonames.org/export/dump/CH.zip
        # unzip -j CH.zip -d inputs/ CH.txt
        cities = extract_cities.main(f"inputs/{MAP_INFO['country_code']}.txt", discretized_extent)
        cached.put(cities)


# with open("japan_cities_in_extent.json") as f:
    # cities = json.load(f)

# plot data
f, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(water_mask, origin="lower", cmap="Blues")

cities = list(sorted(cities, key=lambda c: c["population"], reverse=True))

# TODO: go largest -> smallest, accept each only if clear space around

@dataclass
class Town:
    id: int
    name: str
    x: int
    y: int
    pop: int

towns = []

MIN_TOWN_DIST = 4
TOWN_DIST_FACTOR = 0.05
TOWN_DISPLAY_SCALE = 10
MOUNTAIN_THR = MAP_INFO.get("mountain_threshold", 120)  # meters

for c in cities:
    if c["population"] < MAP_INFO.get("pop_threshold", 10_000):
        continue

    pop = c["population"] // 1000
    x = (c["longitude"] - discretized_extent[0][1]) / (discretized_extent[1][1] - discretized_extent[0][1]) * water_mask.shape[1]
    y = (c["latitude"] - discretized_extent[0][0]) / (discretized_extent[1][0] - discretized_extent[0][0]) * water_mask.shape[0]
    x = int(np.round(x))
    y = int(np.round(y))
    # ax.text(s=f'{c["asciiname"]} ({pop}k)', x=x, y=y)

    if y >= height.shape[0] or x >= height.shape[1] or height[y, x] < 0 or height[y, x] > MOUNTAIN_THR:
        print(f"Reject {c['asciiname']}: mountainous region or outside boundaries")
        continue

    too_close = False

    for t in towns:
        min_dist = MIN_TOWN_DIST + (np.sqrt(t.pop) + np.sqrt(pop)) * TOWN_DIST_FACTOR
        if (t.x - x) ** 2 + (t.y - y) ** 2 < min_dist ** 2:
            print(f"Reject {c['asciiname']} ({pop}k) too close to {t.name} ({t.pop}k) min_dist={min_dist:.1f} actual_dist={( (t.x - x) ** 2 + (t.y - y) ** 2 )**0.5:.1f}")
            too_close = True
            break

    if too_close:
        continue

    towns.append(Town(id=len(towns), name=c["asciiname"], x=x, y=y, pop=pop))
    ax.text(s=f'{c["asciiname"]} ({pop}k)', x=x, y=y)

    area = np.sqrt(pop) * TOWN_DISPLAY_SCALE       # MPL uses points squared as the unit of size for scatter
    ax.scatter(x, y, s=area, c="red", alpha=0.5, edgecolors="black", linewidths=0.5)

print(f"{len(towns)=}")
plt.savefig(f"outputs/{MAP_NAME}.png")

In [ ]:
import math

K = 1.7

traffic_directed = {}

for i, t1 in enumerate(towns):
    weights = {}

    for j, t2 in enumerate(towns):
        if j == i:
            continue

        dist_sq = (t1.x - t2.x) ** 2 + (t1.y - t2.y) ** 2
        weights[j] = t2.pop * math.pow(dist_sq, -K / 2)

    weight_sum = sum(weights.values())

    for j, t2 in enumerate(towns):
        if j == i:
            continue

        traffic = t1.pop * weights[j] / weight_sum
        traffic_directed[(i, j)] = traffic


# note: if really need to squeeze bits, demand could be encoded logarithmically
demand_table = [(int(traffic_directed.get((i, j), 0) +
                     traffic_directed.get((j, i), 0)), i, j) for j in range(len(towns))
                                                             for i in range(len(towns))]
demand_table = [(d, t1, t2) for d, t1, t2 in demand_table if t1 < t2 and d > 0]
demand_table = sorted(demand_table, reverse=True)

with open("/projects/STAK/japan.json", "wt") as f:
    json.dump({
        "water_mask": np.flip(water_mask, axis=0).tolist(),
        "towns": [t.__dict__ | dict(y = water_mask.shape[0] - t.y - 1) for t in towns],
        "demand_table": demand_table,
    }, f)

In [ ]:
# Create tile map in Tiled JSON format
tilemap = {
    "compressionlevel": -1,
    "height": water_mask.shape[0],
    "width": water_mask.shape[1],
    "infinite": False,
    "layers": [
        {
            # Terrain layer
            "data": [3 if height[y, x] > MOUNTAIN_THR else (2 if water_mask[y,x] > 0 else 1)
                    for y in range(water_mask.shape[0]-1, -1, -1)
                    for x in range(water_mask.shape[1])],
            "height": water_mask.shape[0],
            "width": water_mask.shape[1],
            "id": 1,
            "name": "Terrain",
            "opacity": 1,
            "type": "tilelayer",
            "visible": True,
            "x": 0,
            "y": 0
        },
        {
            # Towns layer
            "draworder": "topdown",
            "id": 2,
            "name": "Towns",
            "objects": [
                {
                    "gid": 4,  # Town tile ID
                    "height": 16,
                    "id": t.id + 1,
                    "name": t.name,
                    "properties":[
                            {
                            "name":"population",
                            "type":"int",
                            "value":t.pop,
                            }],
                    "rotation": 0,
                    "visible": True,
                    "width": 16,
                    "x": t.x * 16,
                    "y": (water_mask.shape[0] - t.y) * 16  # Flip Y coordinate
                }
                for t in towns
            ],
            "opacity": 1,
            "type": "objectgroup",
            "visible": True,
            "x": 0,
            "y": 0
        }
    ],
    "nextlayerid": 3,
    "nextobjectid": len(towns) + 1,
    "orientation": "orthogonal",
    "renderorder": "right-up",
    "tiledversion": "1.11.2",
    "tileheight": 16,
    "tilewidth": 16,
    "tilesets": [
        {
            "columns": 4,
            "firstgid": 1,
            "image": "tileset.png",
            "imageheight": 16,
            "imagewidth": 64,
            "margin": 0,
            "name": "tileset",
            "spacing": 0,
            "tilecount": 4,
            "tileheight": 16,
            "tilewidth": 16
        }
    ],
    "type": "map",
    "version": "1.10"
}

# Save to file
with open("japan.tmj", "w") as f:
    json.dump(tilemap, f)